In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

### web scraping (escolher condição e nº de genes)

In [2]:
import time
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

import requests, sys, json
content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    #print (a)

import re
existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
#print(n_genes)

if n_genes == ['']:
    print ("Não encontramos resultado para a sua pesquisa")
else:
    print (n_genes)

    


escolha o que quer pesquisar: IRS
escolha o nº de genes que quer obter (máximo 20): 5
['8223391', '64880', '3376', '45785', '408438']


## Tabela "Gene"

### Buscar ids do ncbi:

In [5]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    print(info.id)

AL329769.1
X61083.1
U02356.1


### Buscar Links dos genes:

In [ ]:
import time
def url_get_id(i):
    url_list= [ ]
    for id in n_genes:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
url_get_id(n_genes)

### Buscar organismos: (novo)

In [13]:
Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Organismos.append(info.annotations['organism'])
print(Organismos)

['Tetraodon nigroviridis', 'Sinorhizobium meliloti', 'Profundulus guatemalensis']


### Buscar seq dos genes:

In [19]:
#Biopython
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:50],'...',info.seq[-10:])
    print()
    seq.append(f'{info.seq[0:50]}...{info.seq[-10:]}')
    

AL329769.1
GGGACAGATCTGGGGGACAGAGCCAGGGGACGGATCCGGGGGACAGATCT ... CCCCANTCTT

X61083.1
AACTCCTTCCTTGCCGAGATCATGGGGGCTGATGGAGGCGGCCGAAGGCG ... GAGGCCGCAA

U02356.1
CCCCGAGGCCTTCCTGCAGGAAGCTCAGGTCATGAAGAAACTGAGACACG ... GGTTGGAGAC



### Buscar % de Alanina:

In [20]:
Alanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('A')
    perc= (count/len(c)*100)
    Alanina.append(perc)
print (Alanina)

[22.906155398587284, 22.11111111111111, 24.561403508771928]


### Buscar % de Citosina:

In [22]:
Citosina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('C')
    perc= (count/len(c)*100)
    Citosina.append(perc)
print (Citosina)

[29.162462159434916, 27.666666666666668, 24.561403508771928]


### Buscar % de Guanina:

In [23]:
Guanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('G')
    perc= (count/len(c)*100)
    Guanina.append(perc)
print (Guanina)

[27.547931382441977, 31.555555555555554, 23.391812865497073]


### Buscar % de Timina:

In [24]:
Timina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('T')
    perc= (count/len(c)*100)
    Timina.append(perc)
print (Timina)

[16.044399596367306, 18.666666666666668, 27.485380116959064]


### Buscar gene description:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    print(info.id)
    print(info.description)
    description.append(info.description)

### Buscar Data:

In [26]:
data=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    data.append(info.annotations['date'])
print(data)

['01-SEP-2000', '12-JUN-2006', '28-NOV-2001']


## Tabela "CDS"

### Buscar quantidade de cds e a sua localização:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
position=0
pos=[]
for info in records:
    #print(info.id)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            featcds.append(i)
            cds= i
            a= str(cds)
            print (len(a))
            position=i
            pos.append(info.features[position].location)
            print (pos)

## Tabela "Uniprot"

### Buscar Ids de proteinas:

In [ ]:
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
            
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT) # is used below in Uniprot

### Buscar Links de proteinas:

In [ ]:
import time
ID_PROT
def url_get_id_p(a):
    url_list_p=[ ]
    for id_p in ID_PROT:
        url_id_p= "https://www.uniprot.org/uniprotkb?query={}".format(id_p)
        url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(ID_PROT)
print(url_ids_protein)  # but we don't need to save this links

### Buscar id proteina (uniprot):

In [ ]:
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [ ]:
#final version
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)


result = {}
for field in fields:
    result[field] = [get_field_for_id(i, field) for i in ID_PROT]

for field, values in result.items():
#     print(f"Results for {field}")
#     print(values)
    #regex
    for x in values:
        entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
        organism = re.search(r'b\'Organism\\n(.+?(?=\\n\'))', x, re.DOTALL)
        function = re.search( r'b\'Function \[(CC)\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
        location = re.search( r'b\'Subcellular location .{27} (.+?(?=\\n\'))', x, re.DOTALL )
        sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
        if entry:
            e = re.match(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL )
            print(e.group(1))
        elif organism:
            o = re.match(r'b\'Organism\\n(.+?(?=\\n\'))', x, re.DOTALL )
            print(o.group(1))
        elif function:
            f = re.match(r'b\'Function .{15} (.+?(?=\\n\'))', x, re.DOTALL )
            print(f.group(1))
        elif location:
            l = re.match(r'b\'Subcellular location .{27} (.+?(?=\\n\'))', x, re.DOTALL )
            print(l.group(1))
        elif sequence:
            s = re.match(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            print(s.group(1) , len(s.group(1)) )
#         else:
#             print("fail")
#ADD CONDITION FOR EMPY RESULT (MEANS THE ID PROTEIN IS NOT REVIEWED)

## Tabela "Pubmed"

### Buscar artigos:

In [ ]:
Ids=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    print("title:", info.get("TI", "-"))
    print('authors: ', info.get("AU", "-"))
    print("source:", info.get("SO", "-"))
    print("affiliation: ", info.get("AD", "-") ) # FAZER O IF/ELSE PARA O CASO DE NÃO TER AFFILIATION
    print()